# Notebooks pour Allgo
[Allgo](https://allgo.inria.fr/) est un service qui offre une interface web pour des outils en ligne de commande. Il dispense ainsi les utilisateurs d'installer les programmes en question et offre la possibilité de cacher leur code source et modèles (pour les outils ayant besoin d'un entraînement). Il permet donc : 
* D'évaluer des outils de recherche (par des groupes de recherche et des industriels ?)
* D'archiver un environnement d'exécution

## Speads

[Speads](https://allgo.inria.fr/app/speads) est un exemple d'application disponible sur Allgo. Speads segmente une conversation, identifie les locuteurs et tente de déterminer leur genre.

Par exemple, en partant de cette conversation :

In [ ]:
from IPython.display import Audio
Audio("conv1.mp3")

On obtient en sortie le tableau suivant :

In [ ]:
import csv
with open('conv1_speads.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        print(row)

Plutôt qu'un simple tableau, on voudrait obtenir un graphique de ce type:
![graphique](plot_1.png)

Ici par exemple, on voit bien plus facilement qu'on a cinq locuteurs dont deux sont des hommes.

Pour obtenir ce graphique, *et pour d'autres tâches d'analyse* -- il ne s'agit pas que de visualisation--, on a besoin d'un environnement plus puissant que l'UI d'Allgo, ce qui justifie le recours à un notebook type Jupyter. Par exemple, dans les données brutes, le genre du locuteur est mélangé avec son identifiant (comme dans `speaker4_F`), ce qui est problématique.

## Préparer les données

Ici, on corrige le problème évoqué plus haut (genre et identifiant du locuteur mélangés)

In [ ]:
import csv
def preprocess_raw_speads_output(infile, outfile):
    with open(outfile, 'w') as dest:
        dest.write("ID\tGender\tStart\tEnd\n")
        with open(infile, 'r') as source:
            reader = csv.reader(source, delimiter='\t')
            for row in reader:
                dest.write('{}\t{}\t{}\t{}\n'.format(row[0][:-2], row[0][-1:], row[1], row[2]))

In [ ]:
preprocess_raw_speads_output(infile='conv1_speads.tsv', outfile='out/conv1_processed.tsv')

Jetons un oeil aux données après traitement :

In [2]:
import pandas as pd
data = pd.read_csv('out/conv1_processed.tsv', sep='\t')
data


,ID,Gender,Start,End
0,speaker0,F,0.00000,1.09000
1,speaker0,F,1.65000,2.60000
2,speaker1,M,3.07000,4.56000
3,speaker2,M,7.35000,20.74000
4,speaker0,F,20.74000,23.20000
5,speaker3,F,23.20000,24.73000
6,speaker0,F,24.73000,27.33000
7,speaker4,F,29.98000,41.44000
8,speaker3,F,41.44000,43.91000
9,speaker1,M,46.53000,75.32000


## Visualiser la conversation avec Vega-Lite

In [ ]:
import vega
vega.VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
      }
    }
  }
},
data)

... ce qui n'est pas le résultat attendu.

* Première surprise : pas de support stable de Vega-Lite dans JupyterLab (alors qu'il s'agit d'un plugin *core*). Au 3 octobre 2017, la bibliothèque python nécessaire n'est pas disponible par `pip` (et un build manuel échoue).
  - Donc, on ne peut embarquer la visualisation précédente que dans Jupyter Classic, à moins d'utiliser des appels bas niveau ? 
* Deuxième surprise : dans Jupyter Classic, le plugin Vega-Lite ne permet pas les visualisations superposées (*layered*). J'ai ouvert [un ticket](https://github.com/altair-viz/jupyter_vega/issues/38) à ce sujet.
* Apparemment, la version de vega-lite utilisée dans le plugin *core* est la v1 (la version actuelle est la v2).

On peut contourner ce problème avec une approche bas niveau (qu'on aimerait quand même éviter...)

In [10]:
def display_static_conversation(dataframe):
  from IPython.display import display
  import json
  bundle_contents = {
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "data": {
    "values": json.loads(dataframe.to_json(orient='records'))
  },
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
              }
          }
      }
  }
  mime_bundle = {'application/vnd.vegalite.v1+json': bundle_contents}
  display(mime_bundle, raw=True)

display_static_conversation(data)

(on note que vegalite a une version de retard...)

## Montrer la tête de lecture sur la visualisation

Au-delà de la première visualisation (utile mais statique), on pourrait aller plus loin en synchronisant la lecture de l'extrait avec le temps sur le graphique. Cela permettrait de comprendre mieux le comportement de l'outil et les données générées. Par exemple:
* La longueur de la conversation décrite est-elle bien celle de l'extrait audio ?
* Les temps de début et de fin sont-ils les bons ?
* L'outil détecte-t-il un même locuteur sous deux identifiants différents, ou le contraire (moins de locuteurs trouvés que dans la conversation) ?

In [ ]:
def create_tape_vis(id, data, tape_pos):
    pass



### Créer un plugin dédié (option lourde)

On devrait pouvoir contourner ces problèmes en écrivant un plugin dédié. C'est une approche lourde et idéalement, on ne devrait pas avoir à y recourir pour un prototype (pour lequel on utiliserait plutôt [cette approche](https://gist.github.com/minrk/1a1e56a611f1ff1e2645f733bdd0e381) pour lier deux composants) ou un tutoriel, mais seulement pour les cas où un besoin affirmé existe. 

Un plugin permet de garder toute l'interaction dans le navigateur et d'éviter les allers-retours entre le kernel et le navigateur, ainsi que d'avoir une interaction aussi personnalisée que nécessaire.

Le plugin est visible dans le dossier `jupyterlab_speads`. Ici on se contente de montrer la partie python minimale du code (presque tout est fait côté client) et le résultat. *En principe, la partie Python écrite ici devrait être packagée séparément et on devrait seulement invoquer `interactive_conversation()`*.


In [3]:
def make_data_url(filename):
    import base64
    import mimetypes
    with open(filename, 'rb') as f:
        return "data:{};base64,{}".format(mimetypes.guess_type(filename)[0], base64.b64encode(f.read()).decode('ascii'))
    
def make_vis_spec(dataframe):
    import json
    return {
    "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
    "layer": [
        {
            "description": "Conversation timeline",
            "data": {
                "values": json.loads(dataframe.to_json(orient='records'))
            },
            "mark": "bar",
            "encoding": {
                "y": {
                    "field": "ID",
                    "type": "nominal"
                },
                "x": {
                    "field": "Start",
                    "type": "quantitative"
                },
                "x2": {
                    "field": "End",
                    "type": "quantitative"
                },
                "color": {
                    "field": "Gender",
                    "type": "nominal",
                    "scale": {
                        "domain": [
                            "F",
                            "M"
                        ],
                        "range": [
                            "#ff99ff",
                            "#4169e1"
                        ]
                    }
                }
            }
        },
        {
            "description": "Tape head",
            "data": {
                "values": [
                    {
                        "timepos": 42
                    }
                ]
            },
            "mark": "rule",
            "encoding": {
                "x": {
                    "field": "timepos",
                    "type": "quantitative"
                },
                "size": {
                    "value": 2
                },
                "color": {
                    "value": "#22aa12"
                }
            }
        }
    ]
}

def interactive_conversation(soundfile_path, dataframe):
    from IPython.display import display
    bundle_contents = {
        "audio_data": make_data_url(soundfile_path),
        "vis_spec": make_vis_spec(dataframe)
    }
    bundle = {"application/vnd.speads+json": bundle_contents}
    display(bundle, raw=True)

interactive_conversation('conv1.mp3', data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Approche entièrement web (sans notebook)

Pour montrer le résultat attendu, voir l'exemple [ici](web_only/index.html).




## Discussion

En préparant ce carnet, mon idée est de :
* Proposer un scenario / cadre de projet collaboratif pour le SED autour d'Allgo (qui avait été évoqué en réunion)
  - Autrement dit, de proposer une UI plus adaptée pour un outil hébergé sur Allgo, en proposant un déploiement aussi simple que possible ; idéalement, rien à faire du côté d'Allgo, et de notre côté, le carnet serait déployé via [Binder](https://beta.mybinder.org).
* Évaluer l'état actuel d'avancement de JupyterLab sur un cas simple mais pas trivial
* Évaluer la difficulté de proposer une visualisation interactive non-native (ici, la synchro) avec Jupyter/JupyterLab et la qualité du résultat obtenu.

Si un tel environnement est proposé, il doit être assez souple; par exemple permettre de choisir la configuration du serveur Jupyter / JupyterLab (`IOPub data rate exceeded.`). Est-ce le cas avec Binder ?

Remarques 

* Pour l'instant, maintenir un plugin JupyterLab pour une démo n'est pour moi **pas judicieux** (API toujours en évolution, lourdeur de l'approche). Mais cela pourrait être envisageable après la sortie de JupyterLab 1.0, pour des projets où il y a un besoin récurrent.
* Une critique de Jupyter [ici](http://opiateforthemass.es/articles/why-i-dont-like-jupyter-fka-ipython-notebook/).

## Annexe: nettoyage

In [ ]:
def clean():
    import os
    os.system('rm out/*')



In [ ]:
clean()